In [3]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential, Model, Input
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path='./drive/MyDrive/Pneumonia/train/'
test_path='./drive/MyDrive/Pneumonia/test/'

In [ ]:
batch = 64
epochs_ = 5

In [ ]:
def Data(shape_):    
    train_normal = len(os.listdir(train_path+'NORMAL'))
    train_bacterial = len(os.listdir(train_path+'BACTERIAL PNEUMONIA'))
    train_viral = len(os.listdir(train_path+'VIRAL PNEUMONIA'))
    
    test_normal = len(os.listdir(test_path+'NORMAL'))
    test_bacterial = len(os.listdir(test_path+'BACTERIAL PNEUMONIA'))
    test_viral = len(os.listdir(test_path+'VIRAL PNEUMONIA'))
    
    train_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.10)
    test_data_generator = ImageDataGenerator(rescale=1./255)

    train_data = train_data_generator.flow_from_directory(train_path, target_size=shape_, shuffle=False, color_mode="rgb", class_mode="categorical", batch_size = batch, subset="training")
    val_data = train_data_generator.flow_from_directory(train_path, target_size=shape_, shuffle=False, color_mode="rgb", class_mode="categorical", batch_size = batch, subset="validation")
    test_data = test_data_generator.flow_from_directory(test_path, target_size=shape_, shuffle=False, color_mode="rgb", class_mode="categorical", batch_size = 1)

    return [train_data, val_data, test_data]

In [ ]:
def Grafik(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])   
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='lower right')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()

In [ ]:
def KendiAlgoritmam():
    dataset = Data((200,200))
    train_data = dataset[0]
    val_data = dataset[1]
    test_data = dataset[2]

    model = Sequential()
    model.add(Conv2D(96, kernel_size=(3,3), padding = "same", activation="relu", input_shape=(200,200,3)))
    model.add(Conv2D(96, kernel_size=(3,3), padding = "same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128, kernel_size=(3,3), padding = "same", activation="relu"))
    model.add(Conv2D(128, kernel_size=(3,3), padding = "same", activation="relu"))
    model.add(Dropout(0.3))
    model.add(Conv2D(256, kernel_size=(5,5), padding = "same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu"))
    model.add(Conv2D(256, kernel_size=(5,5), padding = "same", activation="relu"))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, kernel_size=(3,3), padding = "valid", activation="relu"))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(3, activation="softmax"))
    
    model.summary()
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    history = model.fit(train_data, steps_per_epoch=train_data.samples//batch, validation_data=val_data, epochs=epochs_, validation_steps=val_data.samples//batch)
    
    loss, accuracy = model.evaluate(test_data)
    print("\n", loss, accuracy)
    
    y_true = test_data.classes
    pred = model.predict_classes(test_data).astype(dtype = "int32")
    
    sns.heatmap(confusion_matrix(y_true, pred), annot= True)
    plt.show()
    
    print("\n", classification_report(y_true, pred))
    
    Grafik(history)

In [ ]:
def AlexNet(train, test):
    dataset = Data((227,227))
    train_data = dataset[0]
    val_data = dataset[1]
    test_data = dataset[2]
    print(train_data)
    
    alexnet=Sequential()
    alexnet.add(Conv2D(96,kernel_size=(11,11),strides=(4,4),activation='relu',input_shape=(227,227,3)))
    alexnet.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    alexnet.add(ZeroPadding2D((2,2)))
    alexnet.add(Conv2D(256,kernel_size=(5,5),activation='relu',strides=(1,1)))
    alexnet.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    alexnet.add(ZeroPadding2D((1,1)))
    alexnet.add(Conv2D(384,kernel_size=(3,3),activation='relu',strides=(1,1)))
    alexnet.add(ZeroPadding2D((1,1)))
    alexnet.add(Conv2D(384,kernel_size=(3,3),activation='relu',strides=(1,1)))
    alexnet.add(ZeroPadding2D((1,1)))
    alexnet.add(Conv2D(256,kernel_size=(3,3),activation='relu',strides=(1,1)))
    alexnet.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    alexnet.add(Flatten())
    alexnet.add(Dense(4096,activation='relu'))
    alexnet.add(Dense(4096,activation='relu'))
    alexnet.add(Dense(196,activation='softmax'))
    alexnet.summary()

    alexnet.compile(loss="categorical_crossentropy",optimizer="adam", metrics=["accuracy"])
    
    history=alexnet.fit(train_data, steps_per_epoch=train_data.samples//32, validation_data=val_data, epochs=epochs_, validation_steps=val_data.samples//32)
    
    loss, accuracy = alexnet.evaluate(test_data)
    print("\n", loss, accuracy)
    
    y_true = test_data.classes
    pred = alexnet.predict_classes(test_data).astype(dtype = "int32")
    
    sns.heatmap(confusion_matrix(y_true, pred), annot= True)
    plt.show()
    
    print("\n", classification_report(y_true, pred))
    
    Grafik(history)

In [ ]:
def VGG16(train, test):
    dataset = Data((224,224))
    train_data = dataset[0]
    val_data = dataset[1]
    test_data = dataset[2]
    
    input_ = Input(shape=(224,224,3))
    vgg16 = Conv2D(64, kernel_size=(3,3), padding = "same", activation="relu")(input_)
    vgg16 = Conv2D(64, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = MaxPooling2D(pool_size=(2,2))(vgg16)
    vgg16 = Conv2D(128, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(128, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = MaxPooling2D(pool_size=(2,2))(vgg16)
    vgg16 = Conv2D(256, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(256, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(256, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(256, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = MaxPooling2D(pool_size=(2,2))(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = MaxPooling2D(pool_size=(2,2))(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = Conv2D(512, kernel_size=(3,3), padding = "same", activation="relu")(vgg16)
    vgg16 = MaxPooling2D(pool_size=(2,2))(vgg16)
    vgg16 = Flatten()(vgg16)
    vgg16 = Dense(1024, activation="relu")(vgg16)
    vgg16 = Dense(512, activation="relu")(vgg16)
    output_ = Dense(2, activation="softmax")
    
    vgg16_model=Model(inputs=input_, outputs=output_)

    vgg16_model.summary()
    
    vgg16_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    history = vgg16_model.fit(train_data, steps_per_epoch=train_data.samples//batch, validation_data=val_data, epochs=epochs_, validation_steps=val_data.samples//batch)
    
    loss, accuracy = vgg16_model.evaluate(test_data)
    print("\n", loss, accuracy)
    
    y_true = test_data.classes
    pred = vgg16_model.predict_classes(test_data).astype(dtype = "int32")
    
    sns.heatmap(confusion_matrix(y_true, pred), annot= True)
    plt.show()
    
    print("\n", classification_report(y_true, pred))
    
    Grafik(history)

In [ ]:
def Inception(train, test):
    dataset = Data((299,299))
    train_data = dataset[0]
    val_data = dataset[1]
    test_data = dataset[2]
    
    transfer_inception=InceptionV3(weights="imagenet", input_shape=(299,299,3),include_top=False) #weights='imagenet'
    transfer_inception.trainable=False
    
    flat=Flatten()(transfer_inception.output)
    dense=Dense(512,activation='relu')(flat)
    out=Dense(3,activation='softmax')(dense)
    model=Model(inputs=transfer_inception.inputs,outputs=out)

    model.summary()
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    history = model.fit(train_data, steps_per_epoch=train_data.samples//batch, validation_data=val_data, epochs=epochs_, validation_steps=val_data.samples//batch)
    
    loss, accuracy = model.evaluate(test_data)
    print("\n", loss, accuracy)
    
    y_true = test_data.classes
    pred = model.predict_classes(test_data).astype(dtype = "int32")
    
    sns.heatmap(confusion_matrix(y_true, pred), annot= True)
    plt.show()
    
    print("\n", classification_report(y_true, pred))
    
    Grafik(history)

In [ ]:
KendiAlgoritmam()
#AlexNet()
#VGG16()
#InceptionV3()